In [1]:
import cv2

In [11]:


import cv2
import numpy as np

# Load and preprocess the image
image = cv2.imread("coins2.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

scale_factor = 700 / max(image.shape[:2])  # Scale based on the longest dimension
# if scale_factor < 1:  # Only resize if the image is large
image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor)
gray = cv2.resize(gray, (0, 0), fx=scale_factor, fy=scale_factor)

# Apply Gaussian Blur
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Use Adaptive Thresholding instead of Canny
thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                               cv2.THRESH_BINARY_INV, 11, 2)

# Find contours
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours based on circularity and area
coin_contours = []
for cnt in contours:
    perimeter = cv2.arcLength(cnt, True)
    area = cv2.contourArea(cnt)

    if perimeter == 0:  # Avoid division by zero
        continue

    # Calculate circularity
    circularity = 4 * np.pi * (area / (perimeter ** 2))

    # Coins should be fairly circular (circularity close to 1) and have a reasonable area
    if 0.7 < circularity < 1.2 and area > 500 * (scale_factor ** 2):  
        coin_contours.append(cnt)

# Draw detected coins
cv2.drawContours(image, coin_contours, -1, (0, 255, 0), 2)

# Show results
print(f"Total Coins Detected: {len(coin_contours)}")
cv2.imshow("Detected Coins", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Total Coins Detected: 3


In [10]:
def show(img,time=2.5):
    cv2.imshow("Detected Coins", img)
    cv2.waitKey(time*1000)
    cv2.destroyAllWindows()

In [ ]:
def process(path):
    image = cv2.imread(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    scale_factor = 700 / max(image.shape[:2])
    image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor)
    gray = cv2.resize(gray, (0, 0), fx=scale_factor, fy=scale_factor)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV, 11, 2)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    coin_contours = []
    for cnt in contours:
        perimeter = cv2.arcLength(cnt, True)
        area = cv2.contourArea(cnt)
        if perimeter:
            circularity = 4 * np.pi * (area / (perimeter ** 2))
            if 0.7 < circularity < 1.2 and area > 500 * (scale_factor ** 2):  
                coin_contours.append(cnt)
    cv2.drawContours(image, coin_contours, -1, (0, 255, 0), 2)
    show(image)
    print(len(coin_contours))


In [ ]:

proce